# 文書のベクトル化

 - 頻度ベクトル(Bag-of-Words)
 - 二値ベクトル
 - TF-IDFベクトル(頻度ベクトルの各要素に重み付けしたベクトル)

In [41]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## BoW(Bag-of-Words)ベクトル
BoWベクトルの背景にある考え方

> 1. 文書の集合全体から、たとえば単語という一意な**トークン**(token)からなる**語彙**(vocabulary)を作成する
> 2. 各文書での各単語の出現回数を含んだ特徴ベクトルを構築する

構築されたBoWベクトルの大半の成分は0になるため、**疎ベクトル**(sparse vector)と呼ばれる。



$$
\boldsymbol {x} = \left( \mathrm {tf} \left( t_{1}, d \right) , \dots, \mathrm {tf} \left( t_{i}, d \right), \dots, \mathrm {tf} \left( t_{n}, d \right) \right)
$$

 - $\mathrm {tf} \left( t_{i}, d \right)$ : 文書$d$における単語$t$の出現回数

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two',
])
bag = count_vectorizer.fit_transform(docs)

In [32]:
word_freq_ser = pd.Series(
    bag.toarray().sum(axis=0),
    index=count_vectorizer.get_feature_names()
)

word_freq_ser.sort_values()

two        1
and        2
one        2
shining    2
sun        2
sweet      2
weather    2
the        4
is         5
dtype: int64

In [37]:
bag_of_word_df = pd.DataFrame(
    bag.toarray(),
    columns=count_vectorizer.get_feature_names()
)
bag_of_word_df

,and,is,one,shining,sun,sweet,the,two,weather
0,0,1,0,1,1,0,1,0,0
1,0,1,0,0,0,1,1,0,1
2,2,3,2,1,1,1,2,1,1


---
## TF-IDF(Term Frequency-Inverse Document Frequency)ベクトル
BoWベクトルの各要素は、"文書$d$に特定の単語$t_{i}$が何回現れるか、その回数を数えた"だけの単純なものでした。このベクトルの問題は文書中に特定の単語が多く存在すればするほど、その単語に対する重要度が高くなるので、「です」「と」などといったどの文書にも出現しやすい単語の重要度が高くなってしまいます。

これらの問題は次の方法によって解決します。

”ある単語に対して、対象の文書中で出現した回数をカウントするのに加えて、その単語が**他の文書で**どれだけ出現するかをカウントし、その回数で割る（徐算する）”

上記のようにすることで、特定の文書だけで現れやすい単語、つまり、他の文書ではあまり現れない単語には、特徴量として大きな値を設定することができます。

$$
\boldsymbol {x}_{d} = \left( \mathrm {tf-idf} \left( t_{1}, d \right) , \dots, \mathrm {tf-idf} \left( t_{i}, d \right), \dots, \mathrm {tf-idf} \left( t_{}, d \right) \right)
$$


 - $\mathrm {tf-idf} \left( t_{i}, d \right) = \mathrm {tf}\left( t_{i}, d \right) \times \mathrm {idf} \left( t_{i}, d \right)$

$$
\mathrm {idf}\left( t_{i}, d \right) = \log {\frac {n_{d}}{1 + \mathrm {df}\left( t_{i}, d \right)}}
$$

 - $n_{d}$ : 文書の総数
 - $\mathrm {tf} \left( t_{i}, d \right)$ : 文書$d$における単語$t$の出現回数

上記のTF-IDFで対数が使用されているのは、頻度の低い文書に過剰な重みが与えられないようにするためです。

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

np.set_printoptions(precision=2)
tfidf.fit_transform(count_vectorizer.transform(docs)).toarray()

array([[0.  , 0.43, 0.  , 0.56, 0.56, 0.  , 0.43, 0.  , 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.  , 0.56, 0.43, 0.  , 0.56],
       [0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19]])

In [40]:
tfidf_df = pd.DataFrame(
    tfidf.fit_transform(count_vectorizer.transform(docs)).toarray(),
    columns=count_vectorizer.get_feature_names()
)
tfidf_df

,and,is,one,shining,sun,sweet,the,two,weather
0,0.000000,0.433708,0.000000,0.558478,0.558478,0.000000,0.433708,0.000000,0.000000
1,0.000000,0.433708,0.000000,0.000000,0.000000,0.558478,0.433708,0.000000,0.558478
2,0.502386,0.445076,0.502386,0.191039,0.191039,0.191039,0.296718,0.251193,0.191039
